In [88]:
import numpy as np

In [89]:
Loadings = np.array([0.628,0.372,0.9,
                      0.696,0.313,0.9,
                      0.899,-0.050,0,
                      0.779,-0.201,0,
                      0.728,-0.200,0]).reshape((5,3))
Loadings

array([[ 0.628,  0.372,  0.9  ],
       [ 0.696,  0.313,  0.9  ],
       [ 0.899, -0.05 ,  0.   ],
       [ 0.779, -0.201,  0.   ],
       [ 0.728, -0.2  ,  0.   ]])

In [90]:
Loadings = np.array([[0.628, 0.696, 0.899, 0.779, 0.728],
                     [0.372, 0.313, -0.050, -0.201, -0.2]]).T
np.set_printoptions(suppress=True)
print(Loadings)
np.set_printoptions(suppress=False)

[[ 0.628  0.372]
 [ 0.696  0.313]
 [ 0.899 -0.05 ]
 [ 0.779 -0.201]
 [ 0.728 -0.2  ]]


In [91]:
def RotMatPairFct(dim, tht, pr):
    """ 
    LoadingMat: (p, k) Loadings matrix
    Theta: float Theta rotations in radians
    Pr: (i, j) tuple pair, denoting rotation of column i and j.

    Note that G = [cos(tht) sin(tht)
                   -sin(tht) cos(tht)]
    """
    
    mat = np.eye(dim)
    i, j = pr
    mat[i, i] = np.cos(tht)
    mat[i, j] = np.sin(tht)
    mat[j, i] = -np.sin(tht)
    mat[j, j] = np.cos(tht)
    return mat

In [92]:
LambdaMat = Loadings.copy() # Copy for idempotency
ntht = 200
thtV = np.linspace(0, np.pi / 2, num=ntht)
p, k = LambdaMat.shape
pairs = [(i,j) for i in range(k - 1) for j in range(k) if j > i]

num_cycles = 5
for ccl in range(1, num_cycles + 1):
    for pr in pairs:
        PsiV = [0 for _ in range(ntht)]
        
        for i in range(ntht):
            DeltaMat = LambdaMat @ RotMatPairFct(k, thtV[i], pr)

            hVec = np.sqrt(np.sum(DeltaMat ** 2, axis=1))
            
            DMat = DeltaMat / hVec[:,np.newaxis]

            D2Mat = DMat ** 2

            MeanD2Mat = np.repeat(np.mean(D2Mat, axis=0)[:,np.newaxis], p,axis=1).T
            
            PsiV[i] = np.sum((D2Mat - MeanD2Mat) ** 2)
        
        thtEst = thtV[np.argmax(PsiV)]

        print("cycle:", ccl, 
              "theta:", thtEst, 
              "pair:", pr, 
              "objective:", max(PsiV), 
              sep="\t")
        LambdaMat = LambdaMat @ RotMatPairFct(k, thtEst, pr)

cycle:	1	theta:	0.6551562569044042	pair:	(0, 1)	objective:	0.9512730424010297
cycle:	2	theta:	0.0	pair:	(0, 1)	objective:	0.9512730424010297
cycle:	3	theta:	0.0	pair:	(0, 1)	objective:	0.9512730424010297
cycle:	4	theta:	0.0	pair:	(0, 1)	objective:	0.9512730424010297
cycle:	5	theta:	0.0	pair:	(0, 1)	objective:	0.9512730424010297


37.537688442211056